# Pre process

In [1]:
import getPopulation as gp
import preProcess as pp
import pandas as pd
import geopandas as gpd
import numpy as np

area={'north': 33.58, 'south': 33.53,'east': 133.58, 'west': 133.52}

#Get Population in the Area of Study
bbox = pp.createBoundingBox(area=area)
aos = gpd.GeoSeries([bbox]).to_json()
pop = gp.getPopulationArea(39,aos=aos,crs=6690)    
#Create an agentsdb
agentsdb = np.zeros((pop.TotalPop.sum(),5)) #columns=["age","gender","hhType","hhId","Node"])

for i,g in enumerate(pop.geometry.to_list()):
    agentsdb = pp.appendAgents(agentsdb,pop=pop,index=i,poly=g)

last = np.trim_zeros(agentsdb[:,4],'b').shape[0]
agentsdb = agentsdb[:last,:]
np.savetxt("./data/agentsdb.csv", agentsdb, delimiter=",", header="age,gender,hhType,hhId,Node",fmt="%d,%d,%d,%d,%d")

In [2]:
agents = np.loadtxt('./data/agentsdb.csv',delimiter=',')

In [1]:
import pandas as pd

link = pd.read_csv('./data/linksdb.csv')
link.head()

,# number,node1,node2,length,width
0,0,0,1,4,3
1,1,1,2,19,3
2,2,2,3,174,3
3,3,3,4,53,3
4,4,4,5,44,3


In [3]:
import numpy as np
agentsdb = np.loadtxt('./data/agentsdb.csv',delimiter=',')
nrows = agentsdb.shape[0]
rand_ind = np.random.choice(nrows,size=300,replace=False)
new_agentsdb = agentsdb[rand_ind,:]

In [10]:
np.savetxt("./data/nagentsdb.csv", new_agentsdb, delimiter=",", header="age,gender,hhType,hhId,Node",fmt="%d,%d,%d,%d,%d")

1.0

In [ ]:
#which nodes are within pop.geometry

bbox = createBoundingBox(area)
bbox_gdf = gpd.GeoSeries(bbox)
bbox_gdf.set_crs('EPSG:4326',inplace=True)
bbox_gdf = bbox_gdf.to_crs(crs)
poly = shp.geometry.shape(bbox_gdf[0])
shelters = getPrefShelters(pref_code,crs)
gs = gpd.GeoSeries(shelters.geometry)
sh_in_area = shelters[gs.within(poly)]
return sh_in_area
#disaggregate population to nodes


In [19]:
# Download OSM data
import os
import osmnx as ox
import numpy as np
import pandas as pd
import shapely as shp
import geopandas as gpd
import createLinksAndNodes as cln

def createBoundingBox(area={'north': 33.58, 'south': 33.53,'east': 133.58, 'west': 133.52}):
    #create a Bounding Box
    #returns a shapely geometry
    bbox = shp.geometry.box(area["west"], area["south"], area["east"], area["north"])
    return bbox

def createGraph(area={'north': 33.58, 'south': 33.53,'east': 133.58, 'west': 133.52},crs='EPSG:6690',plot=False):
    #NEW AREA (Aug, 2021)
    maprange = area # Degree 1.0 = 111km. 2.22km x 4.44km square.
    # Obtain the roadmap data from OpenStreetMap by using OSMNX 
    G = ox.graph_from_bbox(maprange['north'], maprange['south'], maprange['east'], maprange['west'], network_type='drive',simplify=True)
    G_projected = ox.project_graph(G,to_crs=crs)
    # Simplify topology
    G_simple = ox.simplification.consolidate_intersections(G_projected, tolerance=10, rebuild_graph=True, dead_ends=False, reconnect_edges=True)
    # save the OSM data as Geopackage
    ox.io.save_graph_geopackage(G_simple)
    print("The graph was saved as 'graph.gpkg'!")
    # Draw a map
    if plot:
        ox.plot_graph(G_simple,bgcolor='white',node_color='red',edge_color='black')
    # Create edges file
    edges = gpd.read_file('./data/graph.gpkg',layer='edges')
    edges.to_file('./data/edges.shp')
    print('The edges file was saved as ESRI shapefile')
    #create database of links and edges
    cln.main()
    print('Database created!')
    if plot:
        cln.plotNetwork()
    return G_simple

def getPrefShelters(pref_code = 39,crs='EPSG:6690',filter=True):
    rootfolder = "/Users/erick/ReGID Dropbox/zDATA"
    datafolder = u"PAREA_Hazard_2018/data/世界測地系"
    areafile = f"{pref_code:02d}/PHRP{pref_code:02d}18.shp"
    path = os.path.join(rootfolder, datafolder, areafile)
    shelters_gc = gpd.read_file(path, encoding='shift_jis')
    shelters = shelters_gc.to_crs(crs)
    if filter:
        shelters = shelters[shelters['TUNAMI']==1]
    return shelters

def getAreaShelters(area={'north': 33.58, 'south': 33.53,'east': 133.58, 'west': 133.52},pref_code=39,crs='EPSG:6690'):
    bbox = createBoundingBox(area)
    bbox_gdf = gpd.GeoSeries(bbox)
    bbox_gdf.set_crs('EPSG:4326',inplace=True)
    bbox_gdf = bbox_gdf.to_crs(crs)
    poly = shp.geometry.shape(bbox_gdf[0])
    shelters = getPrefShelters(pref_code,crs)
    gs = gpd.GeoSeries(shelters.geometry)
    sh_in_area = shelters[gs.within(poly)]
    return sh_in_area

def fixLinksDBAndNodesDB(shelters):
    #fixes the nodesdb to add shelters as nodes
    #requires 'shelters' geodataframe
    #the 'nodesdb.csv' was created with 'createLinksAndNodes.py'
    nodesnp = np.loadtxt('./data/nodesdb0.csv',delimiter=',')
    linksdb = np.loadtxt('./data/linksdb0.csv',delimiter=',')
    #create a numpy array for nodesdb
    nodesdb= np.zeros((nodesnp.shape[0] , nodesnp.shape[1]+2))
    nodesdb[:,:3] = nodesnp[:,:3]
    #create a pandas then numpy for shelter coords
    sheltersdb = pd.DataFrame()
    for i,g in zip(shelters.index,shelters['geometry']):
        x,y=g.coords.xy
        sheltersdb.loc[i,'x'],sheltersdb.loc[i,'y']=x[0],y[0]
    sheltersdb = sheltersdb.to_numpy()
    for i in range(sheltersdb.shape[0]):
        x0, y0 = sheltersdb[i,:]
        dist= ( (nodesnp[:,1] - x0)**2 + (nodesnp[:,2] - y0)**2 )**0.5
        indx= np.argmin(dist)
        nodesdb[indx,3] = 1
    nodesdb[:,4] += 1
    np.savetxt("./data/nodesdb.csv", nodesdb, delimiter=",", header="number,coord_x,coord_y,evacuation,reward",fmt="%d,%.6f,%.6f,%d,%d" )
    np.savetxt("./data/linksdb.csv", linksdb, delimiter=",", header="number,node1,node2,length,width",fmt="%d,%d,%d,%d,%d" )
    return
    

In [20]:
# Set Prefecture / Area of Interest / working CRS
pref_code = 39
area = {'north': 33.58, 'south': 33.53,'east': 133.58, 'west': 133.52}
crs = 'EPSG:6690'
# Create a Graph object
G = createGraph(area=area,crs=crs,plot=False)
# Create a Shelter GeoDataframe
shelters = getAreaShelters(area=area,pref_code=pref_code,crs=crs)
# Fix the nodesdb
fixLinksDBAndNodesDB(shelters)

The graph was saved as 'graph.gpkg'!
The edges file was saved as ESRI shapefile
Database created!


In [31]:
linkdsb = pd.read_csv('./data/actionsdb.csv',header=None)
linkdsb.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,0,6,0,16,22,15,21,29,0,0,0,0,0,0,0
1,1,2,1,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2,2,1,0,0,0,0,0,0,0,0,0,0,0
3,3,2,3,2,0,0,0,0,0,0,0,0,0,0,0
4,4,2,4,3,0,0,0,0,0,0,0,0,0,0,0


In [33]:
linkdsb[linkdsb[11]!=0]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
3106,3106,11,4738,4765,4780,4788,4792,3235,4779,4808,4813,4816,4820,0,0


In [28]:
linkdsb[linkdsb['node2']==3106]

,# number,node1,node2,length,width
3235,3235,3105,3106,18,3
4779,4779,4588,3106,22,3
4808,4808,4615,3106,18,3
4813,4813,4620,3106,5,3
4816,4816,4623,3106,22,3
4820,4820,4627,3106,9,3


In [ ]:
import pandas as pd

df = pd.read_csv("./data/edges.csv")
df.head()

In [ ]:
ndf = df[['u','v','length','lanes']]
ndf.head()

In [ ]:
cdf = ndf.rename(columns={'u':'node1','v':'node2','length':'length','lanes':'width'})
cdf.head()

In [ ]:
cdf.width.fillna(0,inplace=True)
cdf.head()

In [ ]:
cdf['#number']=cdf.index

In [ ]:
cdf.head()

In [ ]:
cdf = cdf[['#number','node1','node2','length','width']]
cdf.head()

In [ ]:
int(cdf.width[4])

In [ ]:
for i,j in enumerate(cdf.width):
        try:
                int(j)
        except:
                print(i,int(j[2]))

In [ ]:
cdf['new_width']=[int(i[2]) for i in cdf.width]

In [ ]:
# cdf = cdf.astype(int)
# cdf.to_csv("./data/linksdb.csv",index=False)


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import geopandas as gpd
import pandas as pd
import os

pref = {1: u'北海道', 2: u'青森県', 3: u'岩手県', 4: u'宮城県', 5: u'秋田県',
        6: u'山形県', 7: u'福島県', 8: u'茨城県', 9: u'栃木県', 10: u'群馬県',
        11: u'埼玉県', 12: u'千葉県', 13: u'東京都', 14: u'神奈川県', 15: u'新潟県',
        16: u'富山県', 17: u'石川県', 18: u'福井県', 19: u'山梨県', 20: u'長野県',
        21: u'岐阜県', 22: u'静岡県', 23: u'愛知県', 24: u'三重県', 25: u'滋賀県',
        26: u'京都府', 27: u'大阪府', 28: u'兵庫県', 29: u'奈良県', 30: u'和歌山県',
        31: u'鳥取県', 32: u'島根県', 33: u'岡山県', 34: u'広島県', 35: u'山口県',
        36: u'徳島県', 37: u'香川県', 38: u'愛媛県', 39: u'高知県', 40: u'福岡県',
        41: u'佐賀県', 42: u'長崎県', 43: u'熊本県', 44: u'大分県', 45: u'宮崎県',
        46: u'鹿児島県', 47: u'沖縄県'}
pref_code = 39
rootfolder = "/Users/erick/ReGID Dropbox/zDATA"
# GetAreaCodes
datafolder = u"PAREA_Town_2018/Shape形式/Shape形式/世界測地系"
areafile = f"{pref_code:02d}/A{pref_code:02d}24POL.shp"
path = os.path.join(rootfolder, datafolder, areafile)
area = gpd.read_file(path, encoding='shift_jis')
# GetPopulationData
datafolder = u"PAREA_StatAllforTown_2018/PAREA-Stat.Population"
headerfile = u"ヘッダーファイル/JC0401S0000.csv"
popfile = f"data/{pref_code:02d}/S{pref_code:02d}4JC0401S0000.csv"
pathfile = os.path.join(rootfolder, datafolder, popfile)
pathheader = os.path.join(rootfolder, datafolder, headerfile) 
pop_h = pd.read_csv(pathheader)
pop = pd.read_csv(pathfile, names=pop_h.columns)
colnames={'行政コード':"Val_GovCod", '図形有無F':"FigureF", '分割合算F':"DivRatioF", '秘匿処理F':"ConcealF", 'リザーブ1':"Reserve1", 'リザーブ2':"Reserve2", '指標数':"NoIndicators", '総数(人口)':"TotalPop",'男(人口)':"Male", '女(人口)':"Female", '世帯数':"NoHouseholds", '面積(Ｋ㎡)':"AreaKm2", '人口密度(Ｋ㎡あたり人口)':"PopDensity"}
pop.rename(columns=colnames,inplace=True)
data = pd.merge(pop, area, on="Val_GovCod")
gdata = gpd.GeoDataFrame(data, geometry= data['geometry'], crs="EPSG:4326")

In [ ]:
gdata.head()

In [ ]:
gdata.to_crs(6690,inplace=True)

In [ ]:
gdata.crs

In [ ]:
import geopandas as gpd
import os

gdf = gpd.read_file('../pre/AOS/aos.shp')
gdf.to_crs(6690,inplace=True)
gdf.crs
# gdf_int = gpd.overlay(gdata,gdf,how='intersection')
# ax = gdf_int.plot()
# gdf.plot(ax=ax,facecolor='none',edgecolor='red')

In [ ]:
gdf_int = gpd.overlay(gdata,gdf,how='intersection')
ax = gdf_int.plot()
gdf.plot(ax=ax,facecolor='none',edgecolor='red')